In [1]:
pip install pandas numpy scikit-learn nltk


In [2]:
import pandas as pd
import numpy as np
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

**Loading Dataset**

In [3]:
file_path = "/content/Ai-Medical-Chatbot Dataset.xlsx"
df = pd.read_excel(file_path)
print(df.head())

                                         Description  \
0      Q. What does abutment of the nerve root mean?   
1  Q. What should I do to reduce my weight gained...   
2  Q. I have started to get lots of acne on my fa...   
3  Q. Why do I have uncomfortable feeling between...   
4  Q. My symptoms after intercourse threatns me e...   

                                             Patient  \
0  Hi doctor,I am just wondering what is abutting...   
1  Hi doctor, I am a 22-year-old female who was d...   
2  Hi doctor! I used to have clear skin but since...   
3  Hello doctor,I am having an uncomfortable feel...   
4  Hello doctor,Before two years had sex with a c...   

                                              Doctor  
0  Hi. I have gone through your query with dilige...  
1  Hi. You have really done well with the hypothy...  
2  Hi there Acne has multifactorial etiology. Onl...  
3  Hello. The popping and discomfort what you fel...  
4  Hello. The HIV test uses a finger prick blood ..

**Checking and dropping nulls**

In [4]:
df.isnull().sum()

,0
Description,0
Patient,0
Doctor,0


In [5]:
df.dropna(inplace=True)

In [6]:
df = df[['Patient', 'Doctor']].dropna()

## **Preprocessing Of Text**

In [7]:
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = text.strip()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    return " ".join(words)
df['Patient'] = df['Patient'].apply(clean_text)
df['Patient'].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


,Patient
0,hi doctori wondering abutting abutment nerve r...
1,hi doctor yearold female diagnosed hypothyroid...
2,hi doctor used clear skin since moved new plac...
3,hello doctori uncomfortable feeling middle spi...
4,hello doctorbefore two year sex call girl dark...


**Encoding**

In [8]:

label_encoder = LabelEncoder()
df['Doctor_Label'] = label_encoder.fit_transform(df['Doctor'])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df['Patient'], df['Doctor_Label'], test_size=0.2, random_state=42)

**Traing andtesting using naive bayes pipeline**

In [10]:

model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


**Accuracy Calculations**

In [11]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")


Model Accuracy: 0.33


In [12]:

def chatbot():
    print("🤖 Medical Chatbot: Type your health question or 'exit' to stop.")

    while True:
        user_input = input("You: ")

        if user_input.lower() == "exit":
            print("Chatbot: Take care! Stay healthy. 😊")
            break
        cleaned_input = clean_text(user_input)
        predicted_label = model.predict([cleaned_input])[0]
        response = label_encoder.inverse_transform([predicted_label])[0]

        print(f"Chatbot: {response}\n")

chatbot()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


🤖 Medical Chatbot: Type your health question or 'exit' to stop.
You: what are symptomsaof diabetes?
Chatbot: Hi. I want to assure you not to worry as everything is going to be fine if proper care and treatment is opted in for. I have thoroughly gone through your case and can well understand your genuine health concerns. 1. No, there is not much problem right now as he vomited and also had a lot of water. It is fine because it was only two drops. 2. We usually do not go for emesis (vomiting) for phenol poisoning cases. Because, it is a volatile compound and causes vapors entering the lungs through the airways. 3. As it was only two drops, I do not think it may cause much trouble. If he feels short of breath contact me back, otherwise fine. 1. He should avoid re-exposure. 2. Rule out if there is some sort of suicidal ideation due to depression or anxiety and take him to a psychiatrist for physical evaluation. For further follow up consult a general practitioner online.--->

You: exit
Cha